In [1]:
import pandas as pd
import numpy as np

In [2]:
# Generate the fake data frame in the docs

import pandas.util.testing as tm; tm.N = 3
def unpivot(frame):
    N, K = frame.shape
    data = {'value' : frame.values.ravel('F'),
            'variable' : np.asarray(frame.columns).repeat(N),
            'date' : np.tile(np.asarray(frame.index), K)}
    return pd.DataFrame(data, columns=['date', 'variable', 'value'])
df = unpivot(tm.makeTimeDataFrame())

In [5]:
df

,date,variable,value
0,2000-01-03,A,0.569177
1,2000-01-04,A,-1.028912
2,2000-01-05,A,-0.501496
3,2000-01-03,B,-0.519812
4,2000-01-04,B,-1.937672
5,2000-01-05,B,-2.248196
6,2000-01-03,C,-1.038492
7,2000-01-04,C,-0.772837
8,2000-01-05,C,-0.928293
9,2000-01-03,D,0.675160


## Pivot tables

In [8]:
# Select out everything for variable A 

df[df['variable'] == 'A']

,date,variable,value
0,2000-01-03,A,0.569177
1,2000-01-04,A,-1.028912
2,2000-01-05,A,-0.501496


That allows me to look at the values for `A`, but suppose I wanted a time series with the different variables in the columns? That calls for a **pivot table**. 

In [10]:
df.pivot(index='date', columns='variable', values ='value')

variable,A,B,C,D
date,,,,
2000-01-03,0.569177,-0.519812,-1.038492,0.67516
2000-01-04,-1.028912,-1.937672,-0.772837,0.90000
2000-01-05,-0.501496,-2.248196,-0.928293,0.02234


In [11]:
# What happens if you leave off the `values` attribute

df.pivot(index='date', columns='variable')

value 
 
 
 variable 
 A 
 B 
 C 
 D 
 
 
 date 
 
 
 
 
 
 
 
 
 2000-01-03 
 0.569177 
 -0.519812 
 -1.038492 
 0.67516 
 
 
 2000-01-04 
 -1.028912 
 -1.937672 
 -0.772837 
 0.90000 
 
 
 2000-01-05 
 -0.501496 
 -2.248196 
 -0.928293 
 0.02234

## Stacking and unstacking

In [13]:
tuples = list(zip(*[['bar', 'bar', 'baz', 'baz',
                        'foo', 'foo', 'qux', 'qux'],
                       ['one', 'two', 'one', 'two',
                        'one', 'two', 'one', 'two']]))

index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])
df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=['A', 'B'])
df2 = df[:4]

In [14]:
df2

A 
 B 
 
 
 first 
 second 
 
 
 
 
 
 
 bar 
 one 
 -1.310992 
 -0.083056 
 
 
 two 
 0.319598 
 -1.731091 
 
 
 baz 
 one 
 1.027200 
 -1.203994 
 
 
 two 
 -0.402712 
 -0.049354

In [17]:
# .stack() pivots the column variables into a new set of row labels

stacked = df2.stack()
stacked

first  second   
bar    one     A   -1.310992
               B   -0.083056
       two     A    0.319598
               B   -1.731091
baz    one     A    1.027200
               B   -1.203994
       two     A   -0.402712
               B   -0.049354
dtype: float64

In [18]:
# Takes it back to unpivoted form

stacked.unstack()

A 
 B 
 
 
 first 
 second 
 
 
 
 
 
 
 bar 
 one 
 -1.310992 
 -0.083056 
 
 
 two 
 0.319598 
 -1.731091 
 
 
 baz 
 one 
 1.027200 
 -1.203994 
 
 
 two 
 -0.402712 
 -0.049354

## Melting

In [19]:
cheese = pd.DataFrame({'first' : ['John', 'Mary'],
                       'last' : ['Doe', 'Bo'],
                       'height' : [5.5, 6.0],
                       'weight' : [130, 150]})

In [20]:
cheese

,first,height,last,weight
0,John,5.5,Doe,130
1,Mary,6.0,Bo,150


In [21]:
cheese.melt(id_vars=['first', 'last'])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


What just happened: 
    
- The `id_vars` changed from column variables to identifier variables
- The new variable `variable` was introduced and is set to equal either `height` or `weight` -- i.e. those two column variables are now on the _inside_ of the table collected under the heading `variable`, and their values paired off 

In [22]:
# What if just one id_var?

cheese.melt(id_vars='first')

,first,variable,value
0,John,height,5.5
1,Mary,height,6
2,John,last,Doe
3,Mary,last,Bo
4,John,weight,130
5,Mary,weight,150


In [23]:
# What if different id_vars?

cheese.melt(id_vars=['height', 'weight'])

,height,weight,variable,value
0,5.5,130,first,John
1,6.0,150,first,Mary
2,5.5,130,last,Doe
3,6.0,150,last,Bo


Metaphor: We are "melting down" the DataFrame into its component parts and rearranging the parts. 

`id_vars` tells `.melt()` how to organize the frame, which component parts we are focusing on. 

## Pivot tables

**Difference between `pivot` and `pivot_table`:** `pivot_table` is a generalization of `pivot`. It should be used whenever there is a duplicate value for an index/column pair. 

Example from https://hackernoon.com/reshaping-data-in-python-fa27dda2ff77:

In [34]:
acc = pd.read_csv('accounts.csv')

In [32]:
acc

,date,person,dollars
0,2000-01-03,Michael,200.0
1,2000-01-03,George,500.0
2,2000-01-03,Lisa,450.0
3,2000-01-04,Michael,180.5
4,2000-01-04,George,450.0
5,2000-01-04,Lisa,448.0
6,2000-01-05,Michael,177.0
7,2000-01-05,George,420.0
8,2000-01-05,Lisa,447.0
9,2000-01-06,Michael,150.0


This is "stacked" in the sense that the observations are all stacked on top of each other. Suppose we want to "unstack" this and turn it into a time series where each individual is a column:

In [35]:
acc.columns

Index(['date', 'person', 'dollars'], dtype='object')

In [36]:
acc.pivot(index='date', columns='person', values='dollars')

person,George,Lisa,Michael
date,,,
2000-01-03,500.0,450.0,200.0
2000-01-04,450.0,448.0,180.5
2000-01-05,420.0,447.0,177.0
2000-01-06,300.0,344.6,150.0


This is now "unstacked" -- individual observations are no longer stacked on top of each other. 

Now look at this data frame:

In [37]:
dfc = pd.read_csv('cookies.csv')

In [38]:
dfc

,cookies_sold,date,name
0,1,2000–01–01,George
1,3,2000–01–01,Michael
2,3,2000–01–01,Lisa
3,2,2000–01–01,George
4,4,2000–01–01,Lisa


We can't `pivot` this around dates with names in the columns, because the entry for row "2000-01-01" column "George" has two entries, 1 and 2. (Similarly with Lisa.)

In [40]:
dfc.pivot(index='date',
                       columns='name',
                       values='cookies_sold')

ValueError: Index contains duplicate entries, cannot reshape

So instead we have to use `pivot_table` which uses an aggregator to handle the multiple entries. We can make it aggregate by mean, median, etc. 

In [42]:
# The default aggregator is the mean: 

dfc.pivot_table(index='date',
                             columns='name',
                             values='cookies_sold')

name,George,Lisa,Michael
date,,,
2000–01–01,1.5,3.5,3.0


In [44]:
# We can switch that up though: 

dfc.pivot_table(index="date", columns="name", values="cookies_sold", aggfunc='sum')

name,George,Lisa,Michael
date,,,
2000–01–01,3,7,3


In [46]:
# We can pivot this table in some cases
# This doesn't throw errors but there's a lot of missing data

dfc.pivot(index='name', columns='cookies_sold', values='date')

cookies_sold,1,2,3,4
name,,,,
George,2000–01–01,2000–01–01,None,None
Lisa,None,None,2000–01–01,2000–01–01
Michael,None,None,2000–01–01,None
